# Teste Técnico Axur - Web Scraping e API de Inferência

Este notebook executa as tarefas solicitadas no teste técnico da Axur para estágio:

**Pré-condições:**

- Acesso à internet para realizar scraping e chamadas API.
- Token de autenticação válido (fornecido pela Axur).
- Bibliotecas Python instaladas: `requests`, `beautifulsoup4`, `json`, `base64`.
- Ambiente Python 3.x configurado.

O script realiza as seguintes etapas:
1. Faz o scraping de uma página para baixar uma imagem.
2. Codifica essa imagem em base64 e envia para inferência em um modelo da Microsoft (Florence-2-large).
3. Recebe a resposta do modelo e submete para a plataforma da Axur.


---
### Instruções Adicionais

Em caso de problemas ao executar este notebook, você pode executar o código `.py` presente no repositório GitHub:

[https://github.com/GUS74V0/webScrap--Axur.git](https://github.com/GUS74V0/webScrap--Axur.git)

Basta clonar o repositório e rodar o script Python diretamente no seu ambiente local:

```bash
git clone https://github.com/GUS74V0/webScrap--Axur.git
cd webScrap--Axur
python webScrap.py


In [2]:
!pip install requests beautifulsoup4

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached charset_normalizer-3.4.2-cp313-cp313-win_amd64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
  Using cached soupsieve-2.7-py3-none-any.whl.metadata (4.6 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.2-cp313-cp313-win_amd64.whl (105 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.4.0-py3-none-any.whl (128 kB)
Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
Using cached certifi-2025.4.26-py3-none-any.whl (159 kB)
Using cached soupsieve-2.7-py3-none-any.whl (36 kB)
Using cached typing_extensions-4.13.2-py3-none-any.

In [3]:

import requests
from bs4 import BeautifulSoup
import json
import base64


In [4]:

AUTH_TOKEN = "..."

SCRAPE_URL = "https://intern.aiaxuropenings.com/scrape/c102c692-7fb4-4c91-8bc4-08f28789cf4d"
MODEL_URL = "https://intern.aiaxuropenings.com/v1/chat/completions"
SUBMIT_URL = "https://intern.aiaxuropenings.com/api/submit-response"


## Função `baixar_imagem`

- Realiza o scraping da URL fornecida.
- Busca a primeira tag `<img>` na página.
- Se a imagem estiver em base64, decodifica e salva.
- Se a imagem for uma URL válida, baixa o conteúdo e salva.
- Retorna o caminho do arquivo salvo localmente.




In [5]:

def baixar_imagem():
    print("🔍 Buscando imagem na página...")
    response = requests.get(SCRAPE_URL)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, 'html.parser')
    img_tag = soup.find('img')

    if img_tag is None:
        raise Exception("❌ Nenhuma tag <img> encontrada na página!")

    img_src = img_tag['src']
    print(f"🌐 Conteúdo da tag src: {img_src[:50]}...")

    nome_arquivo = 'imagem.jpg'

    if img_src.startswith('data:image'):
        header, encoded = img_src.split(',', 1)
        imagem = base64.b64decode(encoded)
        with open(nome_arquivo, 'wb') as f:
            f.write(imagem)
        print(f"✅ Imagem em base64 salva como {nome_arquivo}!")

    elif img_src.startswith('http'):
        imagem = requests.get(img_src)
        imagem.raise_for_status()
        with open(nome_arquivo, 'wb') as f:
            f.write(imagem.content)
        print(f"✅ Imagem baixada de URL e salva como {nome_arquivo}!")

    else:
        raise Exception("❌ Formato de imagem não reconhecido!")

    return nome_arquivo


## Função `enviar_imagem`

- Abre a imagem salva e converte seu conteúdo para base64.
- Cria um payload JSON no formato esperado pela API do modelo Florence-2-large.
- Envia a requisição POST com o token de autenticação.
- Trata erros HTTP e imprime o JSON da resposta se bem-sucedido.
- Retorna o JSON da resposta do modelo.



In [6]:

def enviar_imagem(caminho_imagem):
    try:
        with open(caminho_imagem, 'rb') as image_file:
            image_base64 = base64.b64encode(image_file.read()).decode('utf-8')

        image_data_url = f"data:image/jpeg;base64,{image_base64}"

        headers = {
            "Authorization": f"Bearer {AUTH_TOKEN}",
            "Content-Type": "application/json"
        }

        payload = {
            "model": "microsoft-florence-2-large",
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image_url",
                            "image_url": {"url": image_data_url}
                        },
                        {
                            "type": "text",
                            "text": "<DETAILED_CAPTION>"
                        }
                    ]
                }
            ]
        }

        response = requests.post(
            MODEL_URL,
            headers=headers,
            json=payload
        )

        response.raise_for_status()

        resposta_json = response.json()

        print("✅ Resposta do modelo recebida com sucesso!")
        print("🧠 Conteúdo da resposta JSON:")
        print(json.dumps(resposta_json, indent=4, ensure_ascii=False))

        return resposta_json

    except requests.exceptions.HTTPError as e:
        print("❌ Erro na requisição HTTP:", e)
        print("Status Code:", response.status_code)
        print("Response:", response.text)
        raise

    except Exception as e:
        print("❌ Erro inesperado:", e)
        raise


## Função `enviar_resposta`

- Recebe o JSON de resposta do modelo.
- Envia esse JSON para o endpoint de submissão da Axur.
- Trata erros HTTP e imprime a resposta da API.

Pré-condição: o JSON da resposta do modelo deve ser válido e estar no formato correto.


In [7]:

def enviar_resposta(resposta_json):
    print("📤 Enviando resposta para a plataforma da Axur...")
    headers = {
        "Authorization": f"Bearer {AUTH_TOKEN}",
        "Content-Type": "application/json"
    }

    response = requests.post(SUBMIT_URL, headers=headers, json=resposta_json)
    response.raise_for_status()

    print("✅ Resposta submetida com sucesso!")
    print("📨 Resposta da API:", response.text)


## Execução Principal

- Executa as funções em sequência:
  1. Baixa a imagem da página.
  2. Envia a imagem para inferência.
  3. Submete a resposta da inferência para a Axur.

- Captura e imprime exceções que possam ocorrer durante a execução.


In [8]:

try:
    caminho = baixar_imagem()
    resposta = enviar_imagem(caminho)
    enviar_resposta(resposta)
    print("🎯 Pipeline executado com sucesso!")
except Exception as e:
    print("❌ Ocorreu um erro durante a execução:", e)


🔍 Buscando imagem na página...
🌐 Conteúdo da tag src: data:image/jpeg;base64,/9j/4AAQSkZJRgABAQEASABIAAD...
✅ Imagem em base64 salva como imagem.jpg!
✅ Resposta do modelo recebida com sucesso!
🧠 Conteúdo da resposta JSON:
{
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "The image shows a young child, likely between 3-5 years old, standing indoors. The child has short blonde hair with a red bow on the right side. She is wearing a light blue sleeveless top and is holding two cell phones—one in each hand. The outer right hand appears to be an older version of the classic flip phone, decorated with white and black, while the inner hand is an older analogue cellphone. She seems to be gesturing towards something or someone out of focus, and the background includes a home setting with blush-pink walls and white curtains. The child appears curious or engaged, as she leans